# BGCluster dRep
Will cluster the clusters based on Average Nucleotide Identity... and choose the best one based on length.




In [1]:
import os
from Bio import SeqIO
import subprocess
import drep
from drep import d_analyze
import pandas as pd
from shutil import copyfile
from itertools import product 
import numpy as np

import warnings
warnings.filterwarnings("ignore")

## Data dir paths 



In [2]:
data_dir = 'data/AS_data'

In [3]:
# hard-coded Antismash directories

Pat_all_AS = data_dir + '/MH-Pat-all_Masurca.AS/'
Pat_sr_AS = data_dir + '/MH-Pat-sr_spades.AS/'
S1_AS = data_dir + '/MH-s1_L2.2k.fa.AS/'
S2_AS = data_dir + '/MH-s2_L2.2k.fa.AS/'
S3_AS = data_dir + '/MH-s3_L2.2k.fa.AS/'
S5_AS = data_dir + '/MH-s5_L2.2k.fa.AS/'


AS_DIR_list = [S1_AS, S2_AS, S3_AS, S5_AS, Pat_all_AS, Pat_sr_AS]

## Directory of fastas for the bins
bins_dir = '/home/matt/Desktop/final_bins/'


In [4]:
## This function will get all the individual gene clusters, in gbk, from and antismash output dir and return an output dir (BGCs_fastas) of fastas, 
# one fasta file for each gene cluster with a good naming convention.


def BGC_gbks_to_fastas(AS_DIR):

    # The list of individual BGC gbks in an AS output dir, excluding 'final'
    gbks = []
    
    ##Make a results dir
    if not os.path.isdir('data/BGCs_fastas'):
        os.makedirs('data/BGCs_fastas')
        
    
    ## Populate gbks list with the paths to all the appropiate gbk files in the dir
    for file in os.listdir(AS_DIR):
        if file.split('.')[-1] == 'gbk':
            if file.split('.')[-2] != 'final':
                    gbks.append(AS_DIR + file)


    ## For each BGC gbk, write the full Nucleotide sequence to a multi-fasta file.   

    seq_records = []

    for gbk in gbks:

        ## Get a useable name for each BGC gbk in the list, format: assembly-name_clusrer-number ie 'PAT_cluster001'
        ## The will become the headers in the fasta file
        
        gbk_name_split = gbk.split('.') 

        assem_name = gbk_name_split[0].split('/')[-1]
        clust_name = gbk_name_split[-2]

        header = assem_name + '_' + clust_name


        ## Get seq record for the BGC
        records = SeqIO.parse(gbk, "genbank")
        for rec in records:
            rec.id = header ## changer the header for the record to the one generated above

            seq_records.append(rec)


    ## Write the seq_records to a file
    ## Make a name for the output dir 
    assembly_name = seq_records[0].id.split('_')[0]
    
    ## Make directory for results
    
    os.mkdir('data/BGCs_fastas/'+assembly_name)
    
    #get current dir 

    cwd = os.getcwd()

    
    
    ## Write out the results to a named directory inside the output directory
    for rec in seq_records:
        id = rec.id
        seq = rec.seq
        id_file = open(cwd+'/'+'data/BGCs_fastas/' + assembly_name +'/'+id+'.fna', "w")
        id_file.write(">"+str(id)+"\n"+str(seq))
        id_file.close()

    
    
    
    

In [5]:
## Already done
# for AS_DIR in AS_DIR_list:
#     BGC_gbks_to_fastas(AS_DIR)
    

In [6]:
## Walk through the dirs and make a list of all fna's for the dRep run. This will get all *.fna in the 
## cwd recursively, so becareful that there are no other .fna files lying around.
## shoud be 430 files in total  

cwd = os.getcwd()

fnas_list = []
for root, dirs, files in os.walk('data/BGCs_fastas'):

    for name in files:
        if name.endswith('.fna'):
            fnas_list.append(os.path.join(cwd,root,name))

In [7]:
## Run the dRep clustering with the python API


#drep.d_cluster.d_cluster_wrapper('./assemblies_drep_wd_ANImf_25', P_ani = 0.85, S_ani =0.9, S_algorithm = 'ANImf', genomes = fnas_list, COV_THRESH = 0.25)

# plot the outputs
#drep.d_analyze.d_analyze_wrapper('./assemblies_drep_wd_ANImf_25', plots = ['1','2','3','4'])


## Choose the longest BGC from each dRep cluster 

In [8]:
## load in the clustering data table from dRep

clus_df = pd.read_csv('assemblies_drep_wd_ANImf_25/data_tables/Cdb.csv')


## Make a df of cluster lengths and paths

genomes = [] 
lengths = []
paths = []

for fna in fnas_list:
    record = SeqIO.parse(fna, 'fasta')
    
    paths.append(fna)
    for rec in record:
        name = rec.id
        BCG_length = len(rec.seq)
        
    genomes.append(name+'.fna')
    lengths.append(BCG_length)
    

BGC_df = pd.DataFrame( {'genome':genomes, 'length':lengths, 'path':paths})

In [9]:
## merge dataframes to add lenght and path to clus_df

clus_df = clus_df.merge(BGC_df, on='genome').drop(['threshold','cluster_method','comparison_algorithm'], axis =1)


In [10]:
clus_df.head()

,genome,secondary_cluster,primary_cluster,length,path
0,MH-s3_L2_cluster089.fna,1_1,1,3231,/home/matt/Desktop/BGC_Cluter_dRep/data/BGCs_f...
1,MH-s5_L2_cluster047.fna,1_2,1,2438,/home/matt/Desktop/BGC_Cluter_dRep/data/BGCs_f...
2,MH-s3_L2_cluster083.fna,2_1,2,3836,/home/matt/Desktop/BGC_Cluter_dRep/data/BGCs_f...
3,MH-s5_L2_cluster024.fna,2_1,2,27169,/home/matt/Desktop/BGC_Cluter_dRep/data/BGCs_f...
4,MH-s2_L2_cluster096.fna,3_1,3,3320,/home/matt/Desktop/BGC_Cluter_dRep/data/BGCs_f...


In [11]:
## Add the origianl contig id to the data frame


## The antismash dirs mapped to assembly names 
assembly_names = {'MH-Pat-all':Pat_all_AS,
                  'MH-Pat-sr':Pat_sr_AS,
                  'MH-s1': S1_AS,
                  'MH-s2': S2_AS,
                  'MH-s3': S3_AS,
                  'MH-s5': S5_AS}



## Load in the 'geneclusters' file for each assembly antismash dir as a pandas df and overwrite the 'assembly_names' dictionary 
## These .txt files have the clusters in order and the contig info as a tsv
for ass, dirct in assembly_names.items():
    assembly_names[ass] = pd.read_csv(dirct+'geneclusters.txt', sep = '\t', header=None)



def get_contig(cluster_name_string):
    '''Feed this funct a cluster and it will fetch the contig it came from'''
    
    #get the assembly of origin form cluster name to select the right dataframe form thr dict
    name_string_split = cluster_name_string.split('_')
    assembly_name = name_string_split[0] # This will get us the dataframe form the dictonary
    
    ## Get the cluster number to query the dataframe by index
    ## This needs to be an interger...
    clust_number = name_string_split[2].replace('.fna','').replace('cluster','').lstrip('0')
    clust_index = (int(clust_number)-1)
    
    
    contig = assembly_names[assembly_name].iloc[clust_index][1]
    
    return contig

In [12]:
## Add the contig info to the dataframe of BGC
clus_df['contig'] = clus_df['genome'].apply(lambda cluster: get_contig(cluster)) 

In [13]:
## From the contig get bin or unbinned 

##Make a dictionary of contigs for each bin

bins_contigs={} # dictionary of contigs for each bin
for each_bin in os.listdir(bins_dir):
    bins_seqs = SeqIO.parse(bins_dir+each_bin, 'fasta')
    
    bins_contigs[each_bin]= [contig.id for contig in bins_seqs]


In [14]:
## funct to get bin from contig by querying the 'bins_contigs' dict

def contig_to_bin(contig):
    for k, v in bins_contigs.items():
        if contig in v:
            return k
        
    return 'Unbinned'

In [15]:
## Add the binned info to the dataframe of BGC
clus_df['binned'] = clus_df['contig'].apply(lambda contig: contig_to_bin(contig)) 




In [16]:
## Select the representitive BGC. If it was binned it is the rep for that secondary_cluster; else the longest one is chosen


## Group the dataframe by sec_clust then check for a binned cluster
binned_BGCs_list = []
for sec_clus, df in clus_df.groupby(['secondary_cluster'],sort=False):
    if any(df[df['binned']!='Unbinned']):
        df_bins = df[df['binned']!='Unbinned']
        
        #if len(df_bins)>1: ## if there are redundant BGC sequences in the bins, get the longest (this only happens about 5 times)
        #    df_bins = df_bins[df_bins['length']==df_bins['length'].max()].drop_duplicates(subset ='secondary_cluster') ## drop dupes to kill any clusters of the same lenght
            
    binned_BGCs_list.append(df_bins)
           
binned_BGCs_df = pd.concat(binned_BGCs_list)

## manually remove the few redundant clusters with out removing the intra-bin cluster repetes 
binned_BGCs_df = binned_BGCs_df[~binned_BGCs_df['genome'].isin(['MH-Pat-all_Masurca_cluster041.fna', 'MH-s5_L2_cluster052.fna'])]


In [17]:
##This can check for duplicate

# for sc, df in binned_BGCs_df.groupby('secondary_cluster'):
#     if len(df) >1:
#         print(sc)
#         print(df[['genome', 'secondary_cluster','length']])
#         print('')
#         print



In [18]:
## Deal with the clusters that are not represented in a bin.

## filter out the sec_clusters that are binned to get all unbinned BGCs
unbinned_BGC_df = clus_df[~clus_df['secondary_cluster'].isin(binned_BGCs_df['secondary_cluster'])]

##Select the unique longest BGC for each group  
long_Unique_BGCs = unbinned_BGC_df.groupby(['secondary_cluster'],sort=False).apply(lambda g: g[g.length==g.length.max()]).drop_duplicates(subset ='secondary_cluster')['genome'] 

## Reset the unbinned_df to the selected BGCs
unbinned_BGC_df = unbinned_BGC_df[unbinned_BGC_df['genome'].isin(long_Unique_BGCs)]

In [19]:
rep_clusters = pd.concat([binned_BGCs_df,unbinned_BGC_df]) #Stack the binned and unbinned dfs together to make final
rep_clusters['Cluster number'] = np.arange(1, len(rep_clusters) + 1) ## This will generate the cluster number for the final antismash output

In [37]:
## the cluster type to the df

def fna_to_GBK(fasta):
    
    asmby_name_map = {'MH-Pat-all':Pat_all_AS,
                  'MH-Pat-sr':Pat_sr_AS,
                'MH-s1':S1_AS,
                'MH-s2':S2_AS,
                'MH-s3':S3_AS,
                'MH-s5':S5_AS}


    
    
    fasta_split = fasta.split('_')
    assembly_name = fasta_split[0]
    cluster_name = fasta_split[-1].replace('.fna', '')
    
    gbk_list = [gbk for gbk in os.listdir(asmby_name_map[assembly_name]) if 'gbk' in gbk and 'final' not in gbk]
    
    for gbk in gbk_list:
        if gbk.split('.')[-2] == cluster_name:
            return os.path.join(asmby_name_map[assembly_name],gbk)



#funct to query the gbk for the cluster type

def get_bgc_type(bin_name):
    
    gbk_path = fna_to_GBK(bin_name)
 
    gbk = SeqIO.parse(gbk_path, 'genbank')
    for rec in gbk:
        return rec.features[0].qualifiers['product'][0]



In [53]:
a_gbk_path = 'data/AS_data/MH-Pat-all_Masurca.AS/c00001_scf7180...cluster003.gbk'

b_gbk_path = 'data/AS_data/MH-s3_L2.2k.fa.AS/c00001_NODE_1_...cluster083.gbk'

gbk = SeqIO.parse(b_gbk_path, 'genbank')

for rec in gbk:
    if rec.features[1].type == 'cluster':
        
    



cluster


'/home/matt/Desktop/BGC_Cluter_dRep'

In [20]:
##Write to table 
## Not all of the clusters in this output can be 're-antismashed'. This is fixed up in the 'Unique_antismashy notebook'

rep_clusters.drop(['primary_cluster','path'],axis=1).to_csv('Unique_BGC_tab.tsv', sep='\t')

In [24]:
rep_clusters.iloc[0]['path']

'/home/matt/Desktop/BGC_Cluter_dRep/data/BGCs_fastas/MH-s3/MH-s3_L2_cluster083.fna'

In [22]:
## Legacy code...
## get the longest BGC for as the representitve 
#rep_clusters = clus_df.groupby(['secondary_cluster'],sort=False).apply(lambda g: g[g.length==g.length.max()]).drop_duplicates(subset ='secondary_cluster') 



In [23]:
## Make a mulitfasta file with all the rep BGCs

rep_cluster_seq_objs = [SeqIO.parse(path, 'fasta') for path in rep_clusters['path']]
records = []
for seq_object in rep_cluster_seq_objs:
    for record in seq_object:
        records.append(record)

        
SeqIO.write(records,'assemblies_rep_BGC.fasta', 'fasta')

199

In [15]:
## Get the BGCs from the ./final_bins_AS results dir - like befor

# data directory 
final_AS_dir = data_dir + '/final_bins_AS/'


# get the GBKs 
final_gbks = []
for AS_DIR in os.listdir(final_AS_dir):
    for file in os.listdir(final_AS_dir+AS_DIR):
        if file.split('.')[-1] == 'gbk':
            if file.split('.')[-2] != 'final':
                final_gbks.append(final_AS_dir+AS_DIR+'/'+ file)

                
# Make a directroy to put them in
if not os.path.isdir('data/final_bins_BGCs_fastas'):
    os.makedirs('data/final_bins_BGCs_fastas')
    
# Write a fasta file from each GBK and stuff it into the dir that was just made

In [16]:
len(final_gbks)

104

In [17]:
seq_records = []

for gbk in final_gbks:

    ## Generate a unique header for the upcoming seq record
    gbk_name_split = gbk.split('/') 
    assem_name = gbk_name_split[3]
    clust_name = gbk_name_split[4].split('.')[-2]

    header = assem_name + '_' + clust_name
    #print(header)

    ## Get seq record for the BGC
    records = SeqIO.parse(gbk, "genbank")
    for rec in records:
        rec.id = header ## changer the header for the record to the one generated above
        seq_records.append(rec)
        
        
#Write the seq_records to fastas in a dir 

## Write out the results to a named directory inside the output directory
for rec in seq_records:
    id = rec.id
    seq = rec.seq
    id_file = open('data/final_bins_BGCs_fastas'+'/'+id+'.fna', "w")
    id_file.write(">"+str(id)+"\n"+str(seq))
    id_file.close()

In [18]:
## dRep the final_bins BGCs with the assemblies dReped BGCs
# make the list of fasta file to dREP


final_bins_BGC = os.listdir('data/final_bins_BGCs_fastas/')
final_bins_BGC_paths = ['data/final_bins_BGCs_fastas/'+ bcg for bcg in final_bins_BGC]
assemblies_BGC_dRepd_paths = list(rep_clusters['path'])

fnal_ass_combind = final_bins_BGC_paths + assemblies_BGC_dRepd_paths


In [19]:
## Run the clustering wrapper 

#drep.d_cluster.d_cluster_wrapper('./combind_drep_wd_gANI_25', P_ani = 0.95, S_ani =.995, S_algorithm = 'ANImf', genomes = fnal_ass_combind, COV_THRESH = 0.05)
#drep.d_analyze.d_analyze_wrapper('./combind_drep_wd_gANI_25', plots = ['1','2','3','4'])

In [20]:
## Get the list of BGCs that dont cluster with one of the final_bin derive BGCs
comb_clus_df = pd.read_csv('combind_drep_wd_gANI_25/data_tables/Cdb.csv')
g_comb_clus_df = comb_clus_df.groupby('secondary_cluster', sort=False)



In [21]:
## List of unbinned gene cluster!
unbinned_BCGs = []

for sc, df in g_comb_clus_df:
    if all(~df['genome'].isin(final_bins_BGC)):
        unbinned_BCGs = unbinned_BCGs + list(df['genome'])
              
        
binned_BCGs = []

for sc, df in g_comb_clus_df:
    if any(df['genome'].isin(final_bins_BGC)):
        binned_BCGs.append(list(df['genome']))
        

In [22]:
#df = unbinned_BCGs.set_index(['secondary_cluster'])

In [24]:
## cp the unbinnded BGCs GBKs to a results file and do a rename
#make a resutes dir
if not os.path.isdir('Unbinned_GBKs'):
    os.makedirs('Unbinned_GBKs')

for fasta in unbinned_BCGs:
    file_tup = fna_to_GBK(fasta)
    copyfile(file_tup[0], 'Unbinned_GBKs/'+file_tup[1])
    
## write the list of paths to a file

with open('Unbinned_gbks.txt', 'w') as f:
    for item in sorted(os.listdir('Unbinned_GBKs')):
        f.write("%s\n" % item)
        
        

In [44]:
if not os.path.isdir('Unbinned_FNAs'):
    os.makedirs('Unbinned_FNAs')
    
for x,y in product(unbinned_BCGs, fnas_list):
    if x == y.split('/')[-1]:
        copyfile(y, 'Unbinned_FNAs/'+x)
        
